## Vamos implementar um exemplo simples de Self-Attention em Python

Este notebook mostra, em pequenos blocos, como construímos todas as peças necessárias para o mecanismo de self-attention: começamos representando palavras como vetores, definimos as funções auxiliares (softmax e multiplicações matriciais) e, por fim, avaliamos como cada token passa a enxergar os demais dentro da mesma frase. O objetivo é manter a matemática acessível, destacando as etapas fundamentais antes de avançar para arquiteturas maiores como Transformers.


**Passos iniciais**
1. **Transformar as palavras em vetores numéricos (One-Hot Encoding)** — convertemos cada palavra para um vetor binário para que a frase possa ser manipulada com álgebra linear.
2. **Aplicar o mecanismo de Self-Attention ao resultado** — calculamos as matrizes *queries*, *keys* e *values*, estimamos quanto cada palavra deve prestar atenção às demais e geramos uma nova representação contextualizada.
3. **Exibir e interpretar a saída** — imprimimos os vetores finais para observar como a atenção redistribuiu a importância entre os tokens.


### One-Hot Encoding
A codificação one-hot transforma cada palavra em um vetor cujo tamanho é igual ao vocabulário e onde apenas uma posição recebe o valor 1. Essa abordagem simples evita ambiguidade (palavras diferentes nunca compartilham o mesmo vetor) e cria uma base neutra para os próximos cálculos. Apesar de não capturar semântica por si só, ela facilita demonstrar o comportamento da atenção sem depender de embeddings pré-treinados.


In [1]:
import numpy as np

def one_hot_encode(word, vocab):
    vector = np.zeros(len(vocab))
    vector[vocab.index(word)] = 1
    return vector


**Exemplo prático**: considere o vocabulário `['o', 'gato', 'pulou', 'do', 'muro']`. A palavra `gato` ocupa a segunda posição dessa lista, logo seu vetor one-hot torna-se `[0, 1, 0, 0, 0]`. Já `muro` seria `[0, 0, 0, 0, 1]`. Como todos os vetores possuem o mesmo tamanho, conseguimos empilhá-los em uma matriz para representar a frase inteira.


In [2]:
sentence = ["o", "gato", "pulou", "no", "telhado"]

**Vocabulário e ordem**: o dicionário de palavras únicas precisa ser montado antes da codificação e, principalmente, mantido na mesma ordem sempre que reutilizarmos o modelo. Isso garante que o índice da palavra corresponda à mesma coluna do vetor em todas as etapas. Em cenários reais, o vocabulário pode vir de uma lista global, de *tokenizers* treinados ou de subpalavras (BPE, WordPiece).


In [3]:
vocab = list(dict.fromkeys(sentence))

A função softmax converte as pontuações (que podem assumir qualquer valor real) em probabilidades normalizadas, destacando as palavras mais relevantes sem descartar completamente as demais. Ao dividir pelo somatório exponencial, garantimos que todos os pesos fiquem entre 0 e 1 e que a soma seja exatamente 1, condição ideal para interpretar a atenção como uma distribuição de importância.


In [4]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)


A função `self_attention` calcula o quanto cada token deve focar em si mesmo e nos outros tokens. Ela implementa os passos clássicos: gera as matrizes de *queries*, *keys* e *values*, mede similaridades por produtos escalares e reescala os valores pelo softmax para produzir uma combinação ponderada que carrega contexto global da frase.


In [5]:
def self_attention(Q, K, V):
    # Calcula as pontuações de atenção
    scores = np.dot(Q, K.T)
    # Aplica softmax às pontuações
    attention_weights = softmax(scores)
    # Calcula a saída ponderada
    output = np.dot(attention_weights, V)
    return output, attention_weights


- `scores`: resultados do produto `Q @ K^T`, que mede a compatibilidade entre cada par de palavras; quanto maior o score, maior a afinidade.
- `attention_weights`: aplica `softmax` em cada linha de `scores` para obter pesos normalizados, interpretados como "porcentagens de atenção" destinadas aos demais tokens.
- `output`: multiplica os pesos pelos vetores `V`, produzindo uma nova representação em que cada palavra incorpora informações das palavras que recebeu maior atenção.


In [6]:
X = np.array([one_hot_encode(word,vocab) for word in sentence])


In [7]:

# Aplicando Self-Attention
output, attention_weights = self_attention(X, X, X)

A matriz de entrada é composta pela pilha dos vetores one-hot (ou embeddings) de cada palavra, com dimensões `(n_palavras, tamanho_vocab)`. Cada linha descreve um token e cada coluna corresponde a um item do vocabulário. Esse formato facilita aplicar multiplicações matriciais em lote, o que é fundamental para que arquiteturas de atenção sejam eficientes em GPU.


In [8]:
print("Vocabulário")
print(vocab)
print("Entrada (X):")
print(X)
print("\nPesos de Atenção:")
print(attention_weights)
print("\nSaída com Self-Attention Aplicada:")
print(output)

Vocabulário
['o', 'gato', 'pulou', 'no', 'telhado']
Entrada (X):
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

Pesos de Atenção:
[[0.40460968 0.14884758 0.14884758 0.14884758 0.14884758]
 [0.14884758 0.40460968 0.14884758 0.14884758 0.14884758]
 [0.14884758 0.14884758 0.40460968 0.14884758 0.14884758]
 [0.14884758 0.14884758 0.14884758 0.40460968 0.14884758]
 [0.14884758 0.14884758 0.14884758 0.14884758 0.40460968]]

Saída com Self-Attention Aplicada:
[[0.40460968 0.14884758 0.14884758 0.14884758 0.14884758]
 [0.14884758 0.40460968 0.14884758 0.14884758 0.14884758]
 [0.14884758 0.14884758 0.40460968 0.14884758 0.14884758]
 [0.14884758 0.14884758 0.14884758 0.40460968 0.14884758]
 [0.14884758 0.14884758 0.14884758 0.14884758 0.40460968]]


Neste código, estamos manualmente definindo embeddings para palavras, atribuindo-lhes valores que indicam suas relações semânticas simplificadas. Esses embeddings ajudam o modelo a entender o contexto e a importância das palavras na frase ao aplicar mecanismos como o Self-Attention. Em aplicações reais, esses vetores são aprendidos automaticamente a partir de grandes conjuntos de dados.

In [9]:
embeddings = {
    'o': np.array([1., 0., 0.]),
    'gato': np.array([0.8, 0.1, 0.1]),
    'pulou': np.array([0.2, 0.8, 0.2]),
    'no': np.array([0.1, 0.1, 0.8]),
    'telhado': np.array([0.8, 0.1, 0.2])
}

X = np.array([embeddings[word] for word in sentence])

output, attention_weights = self_attention(X, X, X)

print("Sentença: ")
print(sentence)
print("\nPesos de Atenção: ")
print(attention_weights)
print("\nSaída com Self-Attention Aplicada:")
print(output)

Sentença: 
['o', 'gato', 'pulou', 'no', 'telhado']

Pesos de Atenção: 
[[0.28625735 0.23436769 0.12862372 0.11638355 0.23436769]
 [0.25887999 0.22505945 0.15086186 0.13787736 0.22732134]
 [0.16980847 0.18030884 0.28562254 0.18030884 0.18395132]
 [0.16237652 0.17415015 0.19055061 0.28426811 0.1886546 ]
 [0.25345973 0.22256183 0.15068702 0.14623354 0.22705788]]

Saída com Self-Attention Aplicada:
[[0.69860875 0.16141087 0.18914189]
 [0.66474473 0.1797153  0.20844447]
 [0.53637198 0.28295493 0.25619273]
 [0.51915726 0.21714778 0.32067055]
 [0.65791626 0.18013494 0.214792  ]]


### Analisando os pesos de atenção
A matriz acima mostra quanto cada palavra foca nas demais. Observando as maiores probabilidades em cada linha:
- `o` distribui atenção de forma relativamente uniforme, mas foca ligeiramente em `gato` e `telhado` (≈0.23).
- `gato` dá mais peso a `o` (≈0.26), mantendo relevância parecida para `telhado`.
- `pulou` prioriza a si próprio (≈0.29).
- `no` direciona a maior parte para si (≈0.28).
- `telhado` distribui atenção entre `o` e `gato`, reforçando o sujeito da frase.

No geral, os maiores pesos aparecem nas diagonais (palavra prestando atenção em si mesma) e nos pares (`o`↔`gato`, `telhado`↔`gato`), indicando que o modelo considera sujeito e local como elementos-chave para representar o contexto dessa sentença.